# Mongo advance

## 1. Setup

### 1.1. Utils

In [ ]:
from bson import json_util as json
from pymongo.cursor import Cursor
from bson import datetime
import pytz

def format_result(rs):
    return '\t{}'.format(json.dumps(rs, indent=2).replace('\n', '\n\t'))

def date_to_datetime(date):
    return datetime.datetime.combine(date, datetime.datetime.min.time()).replace(tzinfo=pytz.utc)

### 1.2. Make connection

In [ ]:
from pymongo import MongoClient, ASCENDING, DESCENDING, IndexModel

mongo = MongoClient(host='localhost', port=27017, maxPoolSize=5)
print('* mongodb connect success: {}'.format(mongo))

### 1.3. Get or create db and collection

In [ ]:
user_collection = user_collection = mongo['test_db']['user']
print('* user collection is: {}'.format(user_collection))

## 2. Pagination

### 2.1. Limit and skip

#### 2.1.1. Function 1

In [ ]:
r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'},
    {'id': 4, 'name': 'Lily', 'gender': 'F'},
    {'id': 5, 'name': 'AUthur', 'gender': 'M'}
])

with user_collection.find({}, limit=1, skip=2) as rs:
    print('* find result with limit=1, skip=2:\n{}'.format(format_result(rs)))

with user_collection.find({}, limit=3, skip=3) as rs:
    print('\n* find result with limit=3, skip=3:\n{}'.format(format_result(rs)))

with user_collection.find({}, limit=3, skip=10) as rs:
    print('\n* find result with limit=3, skip=10:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

#### 2.1.2. Function 2 

In [ ]:
r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'},
    {'id': 4, 'name': 'Lily', 'gender': 'F'},
    {'id': 5, 'name': 'AUthur', 'gender': 'M'}
])

with user_collection.find({}).limit(1).skip(3) as rs:
    print('* find result with limit=1, skip=2:\n{}'.format(format_result(rs)))

with user_collection.find({}).limit(3).skip(3) as rs:
    print('\n* find result with limit=3, skip=3:\n{}'.format(format_result(rs)))

with user_collection.find({}).limit(3).skip(10) as rs:
    print('\n* find result with limit=3, skip=10:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 2.2. Sort

In [ ]:
r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])

with user_collection.find({}).sort([('id', ASCENDING)]) as rs:
    print('* find result sort by id asc:\n{}'.format(format_result(rs)))

with user_collection.find({}, sort=[('id', DESCENDING)]) as rs:
    print('\n* find result sort by id desc:\n{}'.format(format_result(rs)))

rs = user_collection.find_one({}, sort=[('id', DESCENDING)])
print('\n* find result sort by id desc:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

## 3. Index

- Show which index used by query

In [ ]:
def check_query_with_index(rs):
    explain = rs.explain()
    json_path = ['queryPlanner', 'winningPlan', 'inputStage', 'indexBounds']
    for key in json_path:
        explain = explain.get(key)
        if not explain:
            break

    if not explain:
        return 'None'
    return json.dumps(explain)

### 3.1. Create index

In [ ]:
user_collection.drop_indexes()

r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])


with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))


index_name = user_collection.create_index([('id', ASCENDING), ('name', DESCENDING)])
print('* index {} was created'.format(index_name))


with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

with user_collection.find({'id': {'$gt': 1}, 'name': {'$regex': r'[L]\w+'}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

with user_collection.find({'id': {'$gt': 1}, 'name': {'$regex': r'[L]\w+'}, 'gender': 'M'}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

with user_collection.find({'name': {'$regex': r'[L]\w+'}, 'gender': 'M'}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))


user_collection.delete_many({})

### 3.2. Create indexes

In [ ]:
user_collection.drop_indexes()

r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])


with user_collection.find({'id': {'$gt': 2}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))


index_1 = IndexModel([('id', ASCENDING)])
index_2 = IndexModel([('name', DESCENDING), ('gender', ASCENDING)])

index_name = user_collection.create_indexes([index_1, index_2])
print('* index {} was created'.format(index_name))


with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

with user_collection.find({'id': {'$gt': 1}, 'name': {'$regex': r'L\w*'}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

with user_collection.find({'name': {'$regex': r'L\w*'}, 'gender': {'$in': ['M', 'F']}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

with user_collection.find({'$or': [{'id': {'$gt': 1}}, {'name': {'$regex': r'L\w*'}}]}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))


user_collection.delete_many({})

### 3.3. Drop index

In [ ]:
user_collection.drop_indexes()

r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])

index = [('id', ASCENDING), ('name', DESCENDING)]

index_name = user_collection.create_index(index)
print('* index "{}" was created'.format(index_name))

with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

user_collection.drop_index(index)

with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))
    
    
index_name = user_collection.create_index(index)
print('\n* index "{}" was created'.format(index_name))

with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))
    
user_collection.drop_index(index_name)

with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))
    
    
user_collection.delete_many({})

### 3.4. Index arguments

#### 3.4.1. Unique index

In [ ]:
user_collection.drop_indexes()

r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])

index_name = user_collection.create_index([('id', ASCENDING)], unique=True)
print('* unique index "{}" was created'.format(index_name))

try:
    user_collection.insert_one({'id': 3, 'name': 'Authur', 'gender': 'M'})
except Exception as err:
    print('* cannot insert, reason: "{}"'.format(err))

user_collection.delete_many({})

#### 3.4.2. Index name

In [ ]:
user_collection.drop_indexes()

r = user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])

index_name = user_collection.create_index([('id', ASCENDING)], name='ix_id_asc')
print('* unique index "{}" was created'.format(index_name))

with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))


user_collection.drop_index('ix_id_asc')


with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))


user_collection.delete_many({})

#### 3.4.3. Index weights

In [ ]:
user_collection.drop_indexes()

user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])

index_name = user_collection.create_index([('id', ASCENDING)], name='ix_id_asc', weights=1)
print('* unique index "{}" was created'.format(index_name))

index_name = user_collection.create_index([('name', DESCENDING)], name='ix_name_desc', weights=100)
print('* unique index "{}" was created'.format(index_name))

with user_collection.find({'id': {'$gt': 1}}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))

with user_collection.find({'id': {'$gt': 1}, 'name': 'Lucy'}) as rs:
    print('* query by id with index {}'.format(check_query_with_index(rs)))


user_collection.delete_many({})

## 4. Where operator

### 4.1. Use simple javascript expressions as where condition

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])

condition = '''this.id > {} && this.gender == "{}"'''.format(1, 'F')

with user_collection.find({'$where': condition}) as rs:
    print('* query with condition "{}" is:\n {}'.format(condition, format_result(rs)))


user_collection.delete_many({})

### 4.2. Use complex javascript code as where condition

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])

condition = '''
function filter(obj, nameLength, gender) {
    if (obj.name.length !== nameLength) {
        return false;
    }
    return obj.gender === gender;
}
return filter(this, 4, 'F')'''

with user_collection.find({'$where': condition}) as rs:
    print('* query with condition "{}\n" is:\n {}'.format(condition.replace('\n', '\n\t'), format_result(rs)))


user_collection.delete_many({})